In [3]:
import pandas as pd
import numpy as np

# Import Datasets: PV Output and Customer Load

Note: Need to find data that better suits this scenario
Note2: Need to automate the PV data pull here

In [5]:
#Import Datasets

#PV Output, change to PVWatts automatic based on inputs
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer load
CustomerLoad = pd.read_excel('https://openei.org/datasets/dataset/399852d9-91a3-4dd7-86ba-73dd5388c994/'
                              'resource/c0608a74-1ef8-4583-a4cd-33f0c78e8478/download/lbnlb74electricity.xlsx')
    #Change to be an excel file: CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

# Scenario Inputs: Diesel, Storage, PV, and Electricity Rates

Note: Need to get all this into Tkinter
Note2: Need to add in dynamic approach for lifetime on diesel and storage based on throughput
Note3: Need to add in roundtrip efficiency
Note4: Add in solar costs and multi-year

In [19]:
#Input Data

#Storage
storage_size_kW = float(250)
storage_capex = float(250000)
years_of_use = float(10)

#Storage Financing
tenor = 10
interest_rate = 0.20

#Storage Annual Payment
storage_annual_payment = (storage_capex *(interest_rate * (1 + interest_rate) ** tenor) /
                         ((1 + interest_rate) ** tenor - 1))

#Diesel Generator
diesel_size_kW = float(250)
diesel_capex = float(100000)
diesel_fuel_cost =float(3.5)
diesel_price = float(.40)
    
#Electricity Price - TOU
        #00:01 - 11:00
        #11:01 - 15:00
        #15:01 - 24:00
    
#Electricity Price - Fixed
electricity_price = float(0.13)

59630.6892207148

# Clean Datasets and Create New Dataframe

In [7]:
#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']
EnergyMaster = CustomerLoad

# Create Required Columns for New Dataframe

In [8]:
#Add electricity price
EnergyMaster['electricity_price'] = electricity_price

#Add PV production
EnergyMaster['PV_Output'] = PVOutput

#Add post-load pv production
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#Add post PV load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#Add storage state of charge
EnergyMaster['Storage_SOC'] = EnergyMaster['PV_Output_Post_Load'].cumsum().clip(0,storage_size_kW)

#Add new customer load post storage
EnergyMaster['Storage_SOC_Post_Load'] = (EnergyMaster['Storage_SOC']-EnergyMaster['Customer_Load']).clip_lower(0)

# Create Randomized Grid Outages

In [9]:
#variables
days_per_week_grid_down = 3
hoursdown = 4
percent_downtime = days_per_week_grid_down / (7.0*24/hoursdown)

#calculations
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

# Calculate Combined Price of Energy by Scenario

Note: Storage-only scenario cost is the annual payment lease payment plus electricity price
Note2: Storage and PV cost is the annual lease payment plus the cost of solar plus electricity price

In [10]:
#diesel scenario

EnergyMaster['cost_in_diesel'] = (diesel_price * EnergyMaster['grid_down'])
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']].sum(axis=1)
EnergyMaster['diesel_scenario_price'].fillna(EnergyMaster['cost_in_diesel'],inplace=True)

#storage scenario

# Find Diesel Costs by Year

Group the diesel scenario price by year based on timestamp

In [68]:
#Annual diesel cost

#attempt 1 - EnergyMaster = EnergyMaster.groupby([EnergyMaster['timestamp_hour'].dt.year,['diesel_scenario_price'].sum()
#attempt 2 - EnergyMaster['year'] = EnergyMaster.groupby(EnergyMaster['timestamp_hour'].map(lambda x: x.year))

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method